In [27]:
%matplotlib inline
import mne
import os
from mne.io import concatenate_raws, read_raw_gdf
from mne.channels import make_standard_montage
from mne import rename_channels
from mne import Epochs, pick_types, events_from_annotations
from mne.decoding import CSP
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score
import biosig
import json
import pandas as pd
import numpy as np

In [2]:
sample_data_folder = os.getcwd()
sample_data_raw_files = os.listdir(sample_data_folder)
sample_data_raw_files = [i for i in sample_data_raw_files if '.gdf' in i] 
sample_data_raw_files_train = [i for i in sample_data_raw_files if 'T' in i] 
sample_data_raw_files_test = [i for i in sample_data_raw_files if 'E' in i]
sample_data_raw_files_train.sort()
sample_data_raw_files_test.sort()

In [3]:
raws_train = [read_raw_gdf(f, preload=True) for f in sample_data_raw_files_train]
raws_test = [read_raw_gdf(f, preload=True) for f in sample_data_raw_files_test]
headers_train = [json.loads(biosig.header(f)) for f in sample_data_raw_files_train]
headers_test = [json.loads(biosig.header(f)) for f in sample_data_raw_files_test]
for r in raws_train:
    r.set_channel_types({'EEG:C3':'eeg', 'EEG:Cz':'eeg', 'EEG:C4':'eeg', 'EOG:ch01':'eog', 'EOG:ch02': 'eog',\
                      'EOG:ch03':'eog'})
    r.rename_channels({'EEG:C3':'C3', 'EEG:Cz':'Cz', 'EEG:C4':'C4'})
    ten_twenty_montage = make_standard_montage('standard_1020')
    r.set_montage(ten_twenty_montage)
for r in raws_test:
    r.set_channel_types({'EEG:C3':'eeg', 'EEG:Cz':'eeg', 'EEG:C4':'eeg', 'EOG:ch01':'eog', 'EOG:ch02': 'eog',\
                      'EOG:ch03':'eog'})
    r.rename_channels({'EEG:C3':'C3', 'EEG:Cz':'Cz', 'EEG:C4':'C4'})
    ten_twenty_montage = make_standard_montage('standard_1020')
    r.set_montage(ten_twenty_montage)
dfs_train = [r.to_data_frame() for r in raws_train]
dfs_test = [r.to_data_frame() for r in raws_test]

Extracting EDF parameters from /Users/simonfei/MIdata/B0101T.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 604802  =      0.000 ...  2419.208 secs...


/Users/simonfei/opt/anaconda3/lib/python3.7/site-packages/mne/io/edf/edf.py:1000: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, np.uint8).tolist()[0]


Extracting EDF parameters from /Users/simonfei/MIdata/B0102T.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 552451  =      0.000 ...  2209.804 secs...


/Users/simonfei/opt/anaconda3/lib/python3.7/site-packages/mne/io/edf/edf.py:1000: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, np.uint8).tolist()[0]


Extracting EDF parameters from /Users/simonfei/MIdata/B0103T.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 469010  =      0.000 ...  1876.040 secs...


...

/Users/simonfei/opt/anaconda3/lib/python3.7/site-packages/mne/io/edf/edf.py:1000: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, np.uint8).tolist()[0]


Extracting EDF parameters from /Users/simonfei/MIdata/B0904E.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 466802  =      0.000 ...  1867.208 secs...


/Users/simonfei/opt/anaconda3/lib/python3.7/site-packages/mne/io/edf/edf.py:1000: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, np.uint8).tolist()[0]


Extracting EDF parameters from /Users/simonfei/MIdata/B0905E.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 458570  =      0.000 ...  1834.280 secs...


/Users/simonfei/opt/anaconda3/lib/python3.7/site-packages/mne/io/edf/edf.py:1000: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, np.uint8).tolist()[0]


In [4]:
# headers = []
# for f in sample_data_raw_files:
#     headers.append(json.loads(biosig.header(f)))
#events = [i['EVENT'] for i in headers]

In [5]:
# raws = [read_raw_gdf(f, preload=True) for f in sample_data_raw_files];
# for r in raws:
#     r.set_channel_types({'EEG:C3':'eeg', 'EEG:Cz':'eeg', 'EEG:C4':'eeg', 'EOG:ch01':'eog', 'EOG:ch02': 'eog',\
#                       'EOG:ch03':'eog'})
#     r.rename_channels({'EEG:C3':'C3', 'EEG:Cz':'Cz', 'EEG:C4':'C4'})
#     ten_twenty_montage = make_standard_montage('standard_1020')
#     r.set_montage(ten_twenty_montage)
# dfs = [r.to_data_frame() for r in raws]

Train-Test Split

In [6]:
# raws_train = []
# raws_test = []
# dfs_train = []
# dfs_test = []
# headers_train = []
# headers_test = []
# for i in range(len(sample_data_raw_files)):
#     r = raws[i]
#     f = sample_data_raw_files[i]
#     d = dfs[i]
#     h = headers[i]
#     if f[5] == 'T':
#         raws_train.append(r)
#         dfs_train.append(d)
#         headers_train.append(h)
#     else:
#         raws_test.append(r)
#         dfs_test.append(d)
#         headers_test.append(h)
events_train = [i['EVENT'] for i in headers_train]
events_test = [i['EVENT'] for i in headers_test]

Add Stimuli to FeedbackEvent

In [7]:
stims_train = []
for event in events_train:
    s = []
    for e in event:
        if e['TYP'] == '0x0301' or e['TYP'] == '0x0302':
            s.append(e)
    stims_train.append(s)
    
stims_test = []
for event in events_test:
    s = []
    for e in event:
        #if e['TYP'] == '0x0301' or e['TYP'] == '0x0302':
        if e['TYP'] == '0x030f': #unknown cue
            s.append(e)
    stims_test.append(s)
    
stim_times_train = []
for stims in stims_train:
    st = []
    for s in stims:
        st.append(round(s['POS'] * 1000))
    stim_times_train.append(st)

stim_times_test = []
for stims in stims_test:
    st = []
    for s in stims:
        st.append(round(s['POS'] * 1000))
    stim_times_test.append(st)
    
fbe_train = []
for df in dfs_train:
    fbe = []
    for i in range(len(df)):
        fbe.append(0)
    fbe_train.append(fbe)

for i in range(len(stim_times_train)):
    fbe = fbe_train[i]
    sts = stim_times_train[i]
    for st in sts:
        fbe[round(st/4)] = 1
        
fbe_test = []
for df in dfs_test:
    fbe = []
    for i in range(len(df)):
        fbe.append(0)
    fbe_test.append(fbe)

for i in range(len(stim_times_test)):
    fbe = fbe_test[i]
    sts = stim_times_test[i]
    for st in sts:
        fbe[round(st/4)] = 1

for i in range(len(fbe_train)):
    df = dfs_train[i]
    fbe = fbe_train[i]
    df['FeedBackEvent'] = fbe

for i in range(len(fbe_test)):
    df = dfs_test[i]
    fbe = fbe_test[i]
    df['FeedBackEvent'] = fbe

Save csv

In [8]:
# for df in dfs_train:
#     df.to_csv('data/train/' + f, index = False)
# for df in dfs_test:
#     df.to_csv('data/test/' + f, index = False)
sample_data_raw_csvs_train = [f[:6] + '.csv' for f in sample_data_raw_files_train]
sample_data_raw_csvs_test = [f[:6] + '.csv' for f in sample_data_raw_files_test]
for i in range(len(sample_data_raw_csvs_train)):
    f = sample_data_raw_csvs_train[i]
    dfs_train[i].to_csv('data/train/' + f, index = False)
for i in range(len(sample_data_raw_csvs_test)):
    f = sample_data_raw_csvs_test[i]
    dfs_test[i].to_csv('data/test/' + f, index = False)

Truncate all csv to length 120 and combine them into a train labels and a test labels

In [37]:
labels_folder = 'data/y'
label_files = os.listdir(labels_folder)
label_files = [i for i in label_files if '.csv' in i]
label_files = ['data/y/' + i for i in label_files]
label_files_train = [i for i in label_files if 'T' in i]
label_files_test = [i for i in label_files if 'E' in i]
label_files_train.sort()
label_files_test.sort()
labels_train = [pd.read_csv(f, header = None) for f in label_files_train]
labels_test = [pd.read_csv(f, header = None) for f in label_files_test]

In [38]:
####keep only the first 120 labels in each session####
labels_train = [i[:120] for i in labels_train]
labels_test = [i[:120] for i in labels_test]
######################################################

####Subtract every label by 1 so it's 0/1 instead of 1/2####
labels_train = [i - 1 for i in labels_train]
labels_test = [i - 1 for i in labels_test]
############################################################

In [39]:
labels_train = pd.concat(labels_train)
labels_test = pd.concat(labels_test)

Save labels to 2 csv files

In [40]:
labels_train.to_csv('data/TrainLabels.csv', index = False, header = None)
labels_test.to_csv('data/true_labels.csv', index = False, header = None)